In [2]:
# Imports and Setup
import numpy as np
import pandas as pd

In [3]:
# Step 1: Create Decision Matrix
# Supplier/Contractor alternatives with performance data

alternatives = ["GTS", "ISC", "RCI", "ETP"]

criteria = [
    "Cost per Unit",
    "Delivery Time",
    "Quality Score",
    "Innovation Index",
    "Sustainability Score",
]

# Decision matrix with actual values
decision_matrix_array = np.array(
    [
        [4550, 12, 94.2, 6.5, 72],
        [5230, 8, 91.8, 9.2, 68],
        [5870, 15, 98.5, 5.8, 85],
        [6120, 18, 89.3, 8.7, 96],
    ]
)

# Create decision matrix
decision_matrix = pd.DataFrame(
    decision_matrix_array, index=alternatives, columns=criteria
)

decision_matrix

,Cost per Unit,Delivery Time,Quality Score,Innovation Index,Sustainability Score
GTS,4550.0,12.0,94.2,6.5,72.0
ISC,5230.0,8.0,91.8,9.2,68.0
RCI,5870.0,15.0,98.5,5.8,85.0
ETP,6120.0,18.0,89.3,8.7,96.0


In [4]:
# Criteria information
criteria_info = {
    "Cost per Unit": {"type": "minimize", "unit": "Million PHP"},
    "Delivery Time": {"type": "minimize", "unit": "months"},
    "Quality Score": {"type": "maximize", "unit": "score (0-100)"},
    "Innovation Index": {"type": "maximize", "unit": "score (1-10)"},
    "Sustainability Score": {"type": "maximize", "unit": "score (0-100)"},
}

print("Criteria Information:")
for criterion, info in criteria_info.items():
    print(f"• {criterion}: {info['type']} ({info['unit']})")

print(f"\nAlternatives: {len(alternatives)} suppliers/contractors")
print(f"Criteria: {len(criteria_info)} evaluation factors")

Criteria Information:
• Cost per Unit: minimize (Million PHP)
• Delivery Time: minimize (months)
• Quality Score: maximize (score (0-100))
• Innovation Index: maximize (score (1-10))
• Sustainability Score: maximize (score (0-100))

Alternatives: 4 suppliers/contractors
Criteria: 5 evaluation factors


In [5]:
# Step 2: Define Criteria Weights and Preference Functions

# Criteria weights (must sum to 1)
weights = {
    "Cost per Unit": 0.30,
    "Delivery Time": 0.20,
    "Quality Score": 0.25,
    "Innovation Index": 0.15,
    "Sustainability Score": 0.10,
}

print("Criteria Weights:")
total_weight = 0
for criterion, weight in weights.items():
    print(f"• {criterion}: {weight:.2f} ({weight * 100:.0f}%)")
    total_weight += weight
print(f"Total weight: {total_weight:.2f}")

# Preference function parameters
# For this example, we'll use Linear Preference (Type 5) for all criteria
preference_params = {
    "Cost per Unit": {
        "type": "linear",
        "p": 1000,
    },  # p = 1000 Million PHP threshold
    "Delivery Time": {"type": "linear", "p": 10},  # p = 10 months threshold
    "Quality Score": {
        "type": "linear",
        "p": 10,
    },  # p = 10 score points threshold
    "Innovation Index": {"type": "linear", "p": 4},  # p = 4 points threshold
    "Sustainability Score": {
        "type": "linear",
        "p": 30,
    },  # p = 30 score points threshold
}

print("\nPreference Function Parameters:")
for criterion, params in preference_params.items():
    print(
        f"• {criterion}: {params['type']} with p = {params['p']} {criteria_info[criterion]['unit']}"
    )

print("\nInterpretation:")
print(
    "• Linear preference: gradual increase in preference as performance difference grows"
)
print("• Parameter 'p': difference needed for maximum (strict) preference")

Criteria Weights:
• Cost per Unit: 0.30 (30%)
• Delivery Time: 0.20 (20%)
• Quality Score: 0.25 (25%)
• Innovation Index: 0.15 (15%)
• Sustainability Score: 0.10 (10%)
Total weight: 1.00

Preference Function Parameters:
• Cost per Unit: linear with p = 1000 Million PHP
• Delivery Time: linear with p = 10 months
• Quality Score: linear with p = 10 score (0-100)
• Innovation Index: linear with p = 4 score (1-10)
• Sustainability Score: linear with p = 30 score (0-100)

Interpretation:
• Linear preference: gradual increase in preference as performance difference grows
• Parameter 'p': difference needed for maximum (strict) preference


In [8]:
# Step 3: Implement Preference Functions
def linear_preference(d: float, p: float) -> float:
    """
    Linear preference function (Type 5)
    Returns preference degree between 0 and 1
    """
    if d <= 0:
        return 0.0
    elif d >= p:
        return 1.0
    else:
        return d / p


def calculate_preference_degree(
    val_a: float, val_b: float, criterion: str, maximize: bool = True
) -> float:
    """
    Calculate preference degree of alternative a over alternative b for a given criterion
    """
    if maximize:
        d = val_a - val_b  # Positive if a is better than b
    else:
        d = val_b - val_a  # Positive if a is better than b (lower value)

    # Get preference function parameters
    params = preference_params[criterion]
    p = params["p"]

    # Apply preference function
    if params["type"] == "linear":
        return linear_preference(d, p)
    else:
        raise ValueError(
            f"Preference function type '{params['type']}' not implemented"
        )

In [9]:
# Step 4: Calculate Pairwise Preference Matrices
n_alternatives = len(alternatives)
criteria_list = list(decision_matrix.columns)

# Store preference matrices for each criterion
preference_matrices = {}

print("Computing Pairwise Preferences...")
print("=" * 50)

for criterion in criteria_list:
    print(f"\nCriterion: {criterion}")

    # Determine if criterion should be maximized or minimized
    maximize = criteria_info[criterion]["type"] == "maximize"

    # Initialize preference matrix
    pref_matrix = np.zeros((n_alternatives, n_alternatives))

    # Calculate preferences for all pairs
    for i in range(n_alternatives):
        for j in range(n_alternatives):
            if i != j:  # Don't compare alternative with itself
                val_i = decision_matrix.iloc[i][criterion]
                val_j = decision_matrix.iloc[j][criterion]

                pref_degree = calculate_preference_degree(
                    val_i, val_j, criterion, maximize
                )
                pref_matrix[i][j] = pref_degree

    # Convert to DataFrame for better display
    pref_df = pd.DataFrame(
        pref_matrix, index=alternatives, columns=alternatives
    )

    preference_matrices[criterion] = pref_df

    print(f"Preference Matrix ({criterion}):")
    print(pref_df.round(3))

print(f"\nCalculated preference matrices for {len(criteria_list)} criteria")
print(
    "Note: P(A,B) = preference degree of A over B (0 = no preference, 1 = strict preference)"
)

Computing Pairwise Preferences...

Criterion: Cost per Unit
Preference Matrix (Cost per Unit):
     GTS   ISC   RCI   ETP
GTS  0.0  0.68  1.00  1.00
ISC  0.0  0.00  0.64  0.89
RCI  0.0  0.00  0.00  0.25
ETP  0.0  0.00  0.00  0.00

Criterion: Delivery Time
Preference Matrix (Delivery Time):
     GTS  ISC  RCI  ETP
GTS  0.0  0.0  0.3  0.6
ISC  0.4  0.0  0.7  1.0
RCI  0.0  0.0  0.0  0.3
ETP  0.0  0.0  0.0  0.0

Criterion: Quality Score
Preference Matrix (Quality Score):
      GTS   ISC  RCI   ETP
GTS  0.00  0.24  0.0  0.49
ISC  0.00  0.00  0.0  0.25
RCI  0.43  0.67  0.0  0.92
ETP  0.00  0.00  0.0  0.00

Criterion: Innovation Index
Preference Matrix (Innovation Index):
       GTS  ISC    RCI    ETP
GTS  0.000  0.0  0.175  0.000
ISC  0.675  0.0  0.850  0.125
RCI  0.000  0.0  0.000  0.000
ETP  0.550  0.0  0.725  0.000

Criterion: Sustainability Score
Preference Matrix (Sustainability Score):
       GTS    ISC    RCI  ETP
GTS  0.000  0.133  0.000  0.0
ISC  0.000  0.000  0.000  0.0
RCI  0.433 

In [10]:
# Step 5: Calculate Overall Preference Matrix

# Aggregate preferences across all criteria using weights
overall_preference = np.zeros((n_alternatives, n_alternatives))

print("Aggregating Preferences Using Criteria Weights...")

for criterion in criteria_list:
    weight = weights[criterion]
    pref_matrix = preference_matrices[criterion].values

    # Add weighted contribution of this criterion
    overall_preference += weight * pref_matrix

# Convert to DataFrame
overall_pref_df = pd.DataFrame(
    overall_preference, index=alternatives, columns=alternatives
)

print("\nOverall Preference Matrix π(a,b):")
print(overall_pref_df.round(4))

print("\nInterpretation:")
print("• π(A1,A2) = overall preference degree of A1 over A2")
print("• Higher values indicate stronger preference")
print("• Matrix is asymmetric: π(A,B) ≠ π(B,A)")

# Show some key comparisons
print("\nKey Comparisons:")
for i in range(n_alternatives):
    for j in range(i + 1, n_alternatives):
        alt_i, alt_j = alternatives[i], alternatives[j]
        pref_ij = overall_pref_df.loc[alt_i, alt_j]
        pref_ji = overall_pref_df.loc[alt_j, alt_i]

        if pref_ij > pref_ji:
            stronger = alt_i
            diff = pref_ij - pref_ji
        else:
            stronger = alt_j
            diff = pref_ji - pref_ij

        print(f"• {stronger} preferred over the other by {diff:.3f}")

Aggregating Preferences Using Criteria Weights...

Overall Preference Matrix π(a,b):
        GTS     ISC     RCI     ETP
GTS  0.0000  0.2773  0.3862  0.5425
ISC  0.1812  0.0000  0.4595  0.5483
RCI  0.1508  0.2242  0.0000  0.3650
ETP  0.1625  0.0933  0.1454  0.0000

Interpretation:
• π(A1,A2) = overall preference degree of A1 over A2
• Higher values indicate stronger preference
• Matrix is asymmetric: π(A,B) ≠ π(B,A)

Key Comparisons:
• GTS preferred over the other by 0.096
• GTS preferred over the other by 0.235
• GTS preferred over the other by 0.380
• ISC preferred over the other by 0.235
• ISC preferred over the other by 0.455
• RCI preferred over the other by 0.220


In [11]:
# Step 6: Calculate PROMETHEE Flows and Final Ranking
print("PROMETHEE II Flow Calculations")
print("=" * 50)

# Initialize results dictionary
results = {}

for i, alt in enumerate(alternatives):
    # Positive flow: how much this alternative dominates others
    positive_flow = np.sum(
        [overall_pref_df.iloc[i, j] for j in range(n_alternatives) if i != j]
    ) / (n_alternatives - 1)

    # Negative flow: how much others dominate this alternative
    negative_flow = np.sum(
        [overall_pref_df.iloc[j, i] for j in range(n_alternatives) if i != j]
    ) / (n_alternatives - 1)

    # Net flow: final score (higher is better)
    net_flow = positive_flow - negative_flow

    results[alt] = {
        "Positive_Flow": positive_flow,
        "Negative_Flow": negative_flow,
        "Net_Flow": net_flow,
    }

    print(f"{alt}:")
    print(f"  • Positive Flow φ⁺: {positive_flow:.4f} (dominance over others)")
    print(f"  • Negative Flow φ⁻: {negative_flow:.4f} (dominated by others)")
    print(f"  • Net Flow φ:      {net_flow:.4f} (final score)")
    print()

# Create results DataFrame
results_df = pd.DataFrame(results).T

# Sort by Net Flow (descending - higher is better)
final_ranking = results_df.sort_values("Net_Flow", ascending=False)
final_ranking["Rank"] = range(1, len(final_ranking) + 1)

print("FINAL RANKING - PROMETHEE II Results")
print("=" * 50)
print(
    final_ranking[
        ["Net_Flow", "Positive_Flow", "Negative_Flow", "Rank"]
    ].round(4)
)

print("\nRECOMMENDATION:")
best_alternative = final_ranking.index[0]
best_score = final_ranking.iloc[0]["Net_Flow"]
print(f"Select {best_alternative} (Net Flow: {best_score:.4f})")

# Show ranking interpretation
print("\nRanking Interpretation:")
for i, (alt, row) in enumerate(final_ranking.iterrows()):
    rank = i + 1
    score = row["Net_Flow"]
    if rank == 1:
        print(f"{rank}. {alt}: Best choice (φ = {score:.4f})")
    elif rank == len(final_ranking):
        print(f"{rank}. {alt}: Least preferred (φ = {score:.4f})")
    else:
        print(f"{rank}. {alt}: Rank {rank} (φ = {score:.4f})")

PROMETHEE II Flow Calculations
GTS:
  • Positive Flow φ⁺: 0.4020 (dominance over others)
  • Negative Flow φ⁻: 0.1649 (dominated by others)
  • Net Flow φ:      0.2372 (final score)

ISC:
  • Positive Flow φ⁺: 0.3963 (dominance over others)
  • Negative Flow φ⁻: 0.1983 (dominated by others)
  • Net Flow φ:      0.1981 (final score)

RCI:
  • Positive Flow φ⁺: 0.2467 (dominance over others)
  • Negative Flow φ⁻: 0.3304 (dominated by others)
  • Net Flow φ:      -0.0837 (final score)

ETP:
  • Positive Flow φ⁺: 0.1337 (dominance over others)
  • Negative Flow φ⁻: 0.4853 (dominated by others)
  • Net Flow φ:      -0.3515 (final score)

FINAL RANKING - PROMETHEE II Results
     Net_Flow  Positive_Flow  Negative_Flow  Rank
GTS    0.2372         0.4020         0.1649     1
ISC    0.1981         0.3963         0.1983     2
RCI   -0.0837         0.2467         0.3304     3
ETP   -0.3515         0.1337         0.4853     4

RECOMMENDATION:
Select GTS (Net Flow: 0.2372)

Ranking Interpretation:
